In [143]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import openpyxl

In [144]:
client_id = '8d8462f2b93e4555a4e922c55b085087'
client_secret = '2a59486e2df2434e80e73ff1ea10d563'

In [145]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id, 
    client_secret=client_secret
))

In [159]:
def get_artist_tracks(artist_name):
    results = sp.search(q=artist_name, type="artist", limit=1)
    artist_id = results['artists']['items'][0]['id']

    albums = []
    results = sp.artist_albums(artist_id, album_type="album,single,appears_on,compilation", limit=50)
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])

    album_ids = [album['id'] for album in albums]

    tracks_data = []
    track_id_to_album = {}
    track_id_to_release = {}
    seen_track_ids = set()

    for i in range(0, len(album_ids), 20):
        batch_ids = album_ids[i:i+20]
        albums_info = sp.albums(batch_ids)['albums']

        for album_info in albums_info:
            album_name = album_info['name']
            release_date = album_info['release_date']

            tracks_items = album_info['tracks']['items']
            while album_info['tracks']['next']:
                album_info['tracks'] = sp.next(album_info['tracks'])
                tracks_items.extend(album_info['tracks']['items'])

            for track in tracks_items:
                track_id = track['id']
                if track_id in seen_track_ids:
                    continue
                seen_track_ids.add(track_id)
                track_id_to_album[track_id] = album_name
                track_id_to_release[track_id] = release_date

    track_ids = list(track_id_to_album.keys())
    for i in range(0, len(track_ids), 50):
        batch_ids = track_ids[i:i+50]
        full_tracks = sp.tracks(batch_ids)['tracks']

        for track in full_tracks:
            tracks_data.append({
                "track_name": track['name'],
                "record": track_id_to_album[track['id']],
                "record_type": track['album']['album_type'],
                "release_date": track_id_to_release[track['id']],
                "isrc": track.get('external_ids', {}).get('isrc'),
                "spotify_url": track['external_urls']['spotify'],
                "duration": track['duration_ms'],
                "popularity": track['popularity'],
                "explicit": track['explicit'],
            })

    return tracks_data


In [ ]:
tracks = get_artist_tracks("Opeth")
df = pd.DataFrame(tracks)

In [ ]:
df

,track_name,album,album_type,release_date,isrc,spotify_url,duration,popularity,explicit
0,§1,The Last Will And Testament,album,2024-11-22,DE1KY2402795,https://open.spotify.com/track/0E96gZ9JPOSXFEw...,356733,43,False
1,§2,The Last Will And Testament,album,2024-11-22,DE1KY2402796,https://open.spotify.com/track/4XQ8kSN0WTgyD8f...,333853,41,False
2,§3,The Last Will And Testament,album,2024-11-22,DE1KY2402797,https://open.spotify.com/track/42dvQfMObPlPPx4...,310066,43,False
3,§4,The Last Will And Testament,album,2024-11-22,DE1KY2402798,https://open.spotify.com/track/0XHqEwAU4Nc1s7x...,420320,42,False
4,§5,The Last Will And Testament,album,2024-11-22,DE1KY2402799,https://open.spotify.com/track/5yptKqG1ej3TCfn...,449600,40,False
...,...,...,...,...,...,...,...,...,...
2739,Machine Debaser,Metal Mania,compilation,2020-07-10,NLA320119630,https://open.spotify.com/track/1vqgdTbvrjBUiAh...,255373,0,False
2740,Embrace the Evil,Metal Mania,compilation,2020-07-10,QMGB71400058,https://open.spotify.com/track/1qfxeUtejEzT3f9...,219927,0,True
2741,Love at First Fright,Metal Mania,compilation,2020-07-10,NLA320219870,https://open.spotify.com/track/4Oz71OcIxhSH6yR...,180493,0,False
2742,Honey Bucket,Metal Mania,compilation,2020-07-10,USAT20103379,https://open.spotify.com/track/5bxwnvPzTpyXdJd...,181146,0,False


In [ ]:
df.loc[df['isrc'].str.upper() == 'NLA321090067']

,track_name,album,album_type,release_date,isrc,spotify_url,duration,popularity,explicit
135,The Moor - Live,In Live Concert at the Royal Albert Hall,album,2010-09-20,NLA321090067,https://open.spotify.com/track/1MiyaasunvrYlRO...,732493,17,False


In [ ]:
df['album_type'].unique().tolist()

['album', 'single', 'compilation']

In [ ]:
df.to_excel(r'C:\Users\ansh\Desktop\tracks.xlsx', index=False)